# Assignment 2

SID: 1155107874

Name: LI Yuxin

# Setting Up

In [47]:
import pandas as pd
import numpy as np

In [48]:
stock_data = pd.read_excel("stock data.xlsx", sheet_name="stock prices")
stock_data

,Stock code,5,16,66,388,669,688,700,762,883,...,1044,1113,1211,1299,1810,2313,2318,2628,3968,9999
0,2022-06-30,51.65,92.70,41.00,386.0,81.85,24.80,354.4,3.71,10.36,...,36.85,55.50,314.0,85.05,13.64,95.05,53.35,13.66,52.50,144.1
1,2022-07-04,51.25,92.80,40.85,374.0,85.25,25.05,347.0,3.72,10.28,...,36.75,56.10,315.8,85.25,13.84,91.30,53.50,13.50,51.75,145.3
2,2022-07-05,50.90,93.60,40.95,370.4,87.00,24.60,348.8,3.74,10.40,...,36.55,56.05,313.6,86.00,13.36,91.00,53.30,13.54,51.45,145.1
3,2022-07-06,49.10,93.95,41.00,367.8,87.55,24.50,347.6,3.73,9.90,...,36.35,56.05,317.4,84.60,13.18,90.60,52.15,12.48,50.25,142.3
4,2022-07-07,49.65,93.35,41.20,370.6,87.95,24.40,351.6,3.73,9.84,...,36.60,56.75,325.0,84.95,13.42,89.90,52.50,12.54,48.60,139.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,2022-12-22,48.20,105.50,41.20,337.4,89.10,20.30,323.8,4.68,9.81,...,41.30,48.30,201.6,83.90,11.02,86.70,50.50,12.48,42.20,115.6
123,2022-12-23,47.90,105.90,40.90,340.0,88.25,20.55,320.2,5.00,9.81,...,40.85,48.25,194.1,86.70,10.80,86.30,50.45,12.56,41.65,113.3
124,2022-12-28,48.50,105.20,41.20,342.6,88.90,20.55,326.2,4.87,9.98,...,41.05,48.00,193.0,86.90,11.22,88.60,51.25,13.06,43.30,114.6
125,2022-12-29,48.40,104.80,41.15,340.0,87.45,20.15,335.2,4.82,9.94,...,40.60,48.00,191.0,85.10,10.86,86.50,50.70,13.06,43.20,111.3


In [49]:
index1 = 1113; index2 = 688
s1 = stock_data[index1]; s2 = stock_data[index2]

# Part I: Exotic Option Pricing 

## 1. Volatility and correlation calculation from historical data
### i) Calculate the realized volatilities of stock S1 and stock S2 

In [50]:
def daily_rtn(daily_price): return np.log(daily_price / daily_price.shift(1))

def volatility(rtn):
    mu = np.mean(rtn)
    n = np.count_nonzero(~np.isnan(rtn))
    return np.sqrt(np.sum((rtn-mu)**2) / (n-1)) * np.sqrt(252)

u1 = daily_rtn(s1)
u2 = daily_rtn(s2)

# using own implementation
# sigma1 = volatility(u1)
# sigma2 = volatility(u2)

# using APIs
from pandas.core.frame import DataFrame
u1 = DataFrame(u1); u2 = DataFrame(u2)
sigma1 = (u1.std() * np.sqrt(252)).values[0]
sigma2 = (u2.std() * np.sqrt(252)).values[0]

print(f"Volatility of Stock {index1} and {index2} are {sigma1:.5f} and {sigma2:.5f} respectively")

Volatility of Stock 1113 and 688 are 0.25889 and 0.49836 respectively


### ii) Calculate the correlation coefficient 𝜌12 of stock S1 and stock S2

In [51]:
M_corr = u1.join(u2).corr()
rho12 = M_corr.loc[index1, index2]
print(f'correlation of Stock {index1} and {index2} are {rho12:.5f}')

correlation of Stock 1113 and 688 are 0.35814


## 2. Option pricing with Monte Carlo simulation

### i) Use a Monte Carlo scheme with time steps N = 180, i.e. delta t = T/N = 1/180. Give the answers with: (a) 1000 paths; (b) 10000 paths; (c) 100000 paths; (d) 500000 paths.

In [52]:
N = 180
T = 1
r = 3.75 * 0.01
question = ['a', 'b', 'c', 'd']

def ABoO_payoff(S10, S11, S12, S20, S21, S22):
    B1 = np.max([S11/S10, S21/S20])
    B2 = np.max([S12/S10, S22/S20])
    A = (B1 + B2) * 0.5
    return np.max(A - 1.0, 0)


def Monte_Carlo_2_stocks(S10, S20, sigma1, sigma2, rho12, dt, step, rate):
    s_1 = S10; s_2 = S20
    for _ in range(step):
        x1,x2 = np.random.normal(0,1,2)
        e1 = x1; e2 = rho12 * x1 + x2 * np.sqrt(1 - rho12**2)
        s_1 += s_1 * (rate * dt + sigma1 * e1 * np.sqrt(dt))
        s_2 += s_2 * (rate * dt + sigma2 * e2 * np.sqrt(dt))
    
    return s_1, s_2

    
def option_price(path, s10, s20, T, step, rate):
    payoff_list = np.zeros(path)
    dt = T / step
    for i in range(path):
        s11, s21 = Monte_Carlo_2_stocks(s10, s20, sigma1, sigma2, rho12, dt, step//2, rate) # 1st half year
        s12, s22 = Monte_Carlo_2_stocks(s11, s21, sigma1, sigma2, rho12, dt, step//2, rate) # 2nd half year
        payoff_list[i] = ABoO_payoff(s10, s11, s12, s20, s21, s22)
        
    return payoff_list.mean() * np.exp(-rate*T)

import time

np.random.seed(5718) # for reproduction

paths = [1000, 10000, 100000, 500000]
# paths = [1000, 10000, 100000]
prices = []

for p in paths:
    
    start_time = time.time()
    
    # simulate p times
    price = option_price(p, s1[s1.size-1], s2[s2.size-1], T, N, r)
    prices.append(price)
    
    # print(f"time: {(time.time() - start_time)/60:.8f} min")
    start_time = time.time()


for i in range(len(paths)):
    print(f"({question[i]}) Monte Carlo Simulate {paths[i]:6d} paths, optiion price is {prices[i]:.5f}")

(a) Monte Carlo Simulate   1000 paths, optiion price is 0.18220
(b) Monte Carlo Simulate  10000 paths, optiion price is 0.18903
(c) Monte Carlo Simulate 100000 paths, optiion price is 0.18684
(d) Monte Carlo Simulate 500000 paths, optiion price is 0.18636


### ii) Use a Monte Carlo scheme with two time steps N = 2, delta t1 = delta t2 = 1⁄2. Give the answers with: (a) 1000 paths; (b) 10000 paths; (c) 100000 paths; (d) 500000 paths.

In [54]:
N = 2
T = 1

def Monte_Carlo_2_stocks(S10, S20, sigma1, sigma2, rho12, dt, step, rate):
    s_1 = S10; s_2 = S20
    for _ in range(step):
        x1,x2 = np.random.normal(0,1,2)
        e1 = x1; e2 = rho12 * x1 + x2 * np.sqrt(1 - rho12**2)
        s_1 *= np.exp((rate - sigma1**2 / 2) * dt + sigma1 * e1 * np.sqrt(dt))
        s_2 *= np.exp((rate - sigma2**2 / 2) * dt + sigma2 * e2 * np.sqrt(dt))
    
    return s_1, s_2

    
def option_price(path, s10, s20, T, step, rate):
    payoff_list = np.zeros(path)
    dt = T / step
    for i in range(path):
        s11, s21 = Monte_Carlo_2_stocks(s10, s20, sigma1, sigma2, rho12, dt, step//2, rate) # 1st half year
        s12, s22 = Monte_Carlo_2_stocks(s11, s21, sigma1, sigma2, rho12, dt, step//2, rate) # 2nd half year
        payoff_list[i] = ABoO_payoff(s10, s11, s12, s20, s21, s22)
        
    return payoff_list.mean() * np.exp(-rate*T)


np.random.seed(5718) # for reproduction

paths = [1000, 10000, 100000, 500000]
prices = []

for p in paths:
    # simulate p times
    price = option_price(p, s1[s1.size-1], s2[s2.size-1], T, N, r)
    prices.append(price)


for i in range(len(paths)):
    print(f"({question[i]}) Monte Carlo Simulate {paths[i]:6d} paths, optiion price is {prices[i]:.5f}")

(a) Monte Carlo Simulate   1000 paths, optiion price is 0.18741
(b) Monte Carlo Simulate  10000 paths, optiion price is 0.18256
(c) Monte Carlo Simulate 100000 paths, optiion price is 0.18626
(d) Monte Carlo Simulate 500000 paths, optiion price is 0.18532


# Part II: Pricing of a structured product

In [55]:
K0 = 105 * 0.01
KI = 95 * 0.01
AC = 100 * 0.01
CP1 = 0.8 * 0.01
CP2 = 0.55 * 0.01
sigma_im = 27.1 * 0.01
S0 = 48.05

In [56]:
def auto_call(t, Dc, S, Pc): return t >= Dc and S >= Pc

def knock_in(t, Ds, S, Pk): return t > Ds and S < Pk

def expiry_payoff(ctn_ki, Sm, K, NOM): return NOM if ctn_ki == 0 or Sm >= K else NOM * Sm / K 

def discount_factor(r, t): return np.exp(-r * t)

def Monte_Carlo(s0, sigma, dt, step, rate, Ds, Pk, K, NOM, Dc, Pc, CP, cp_date):
    
    t = 0
    s = s0 # close
    ctn_ki = 0
    next_cp_date = 0
    payoff = 0

    for i in range(step):
        
        t += dt
        e = np.random.normal()
        s *= np.exp((rate - sigma**2 / 2) * dt + sigma * e * np.sqrt(dt)) # close 
        
        if knock_in(t, Ds, s, Pk):
            ctn_ki += 1
        
        if t >= cp_date[next_cp_date]:   
            payoff += NOM * CP * discount_factor(r, t)
            # print(f"At coupon date: {next_cp_date}, t = {t:.5f}, payoff = {payoff}")
            next_cp_date += 1
            
        if auto_call(t, Dc, s, Pc):
            # early terminate
            accured_int = NOM * CP * (t - cp_date[next_cp_date-1]) * 360 / 30 * discount_factor(r,t)
            # print(f">>> terminate by auto call: t = {t}, Dc = {Dc} s = {s}, Pc = {Pc}, i = {i}")
            return payoff + NOM + accured_int, True
            
    # on expire date
    payoff += NOM * CP * discount_factor(r, t)
    payoff += expiry_payoff(ctn_ki, s, K, NOM) * discount_factor(r,t)
    # print(f">>> terminate by expire: KI = {ctn_ki}, s = {s}, K = {K}, i = {i}")
    return payoff, False

def print_ans(questions, paths, fair_prices, profits, autocall_count, NOM, step):
    for i in range(len(paths)):
        q = questions[i]
        path = paths[i]
        price = fair_prices[i]
        profit = profits[i]
        ctn = autocall_count[i]
        print(f"({q}) Monte Carlo Simulate {path:6d} paths: \tProduct fair price: {price:.3f} ({price/NOM*100:.3f}%); \tInvestment bank profit: {profit:.3f} ({profit/NOM*100:.3f}%); \tNum of Autocall: {ctn:6d} ({ctn/path*100:.2f}%)")

### 1. Calculate the fair price of the product F% with coupon per month CP set 1. Give the answers based on (a) 1000; (b) 10000; (c) 100000; and (d) 500000 Monte Carlo paths. In each case, record the number of times the auto-call condition is triggered. If the product is sold at an initial price of P=100%, what is the profit of the investment bank?

In [57]:
NOM = 100000

t = 0
Ds = t
Dc = t + 1/12

K = S0 * K0
Pc = S0 * AC
Pk = S0 * KI

CP_DATE = np.array(range(1,8)) / 12

N = 210
T = 7/12
r = 3.75 * 0.01
P = 100 * 0.01

def fair_price(path, s0, T, step, sigma, rate, Ds, Pk, K, NOM, Dc, Pc, CP, cp_date):
    payoff_list = np.zeros(path)
    dt = T / step
    ac_ctn = 0
    for i in range(path):
        # print("-" * 60)
        payoff_list[i], ac = Monte_Carlo(s0, sigma, dt, step, rate, Ds, Pk, K, NOM, Dc, Pc, CP, cp_date)
        if ac:
            ac_ctn += 1
    
    return payoff_list.mean(), ac_ctn


np.random.seed(5718) # for reproduction

paths = [1000, 10000, 100000, 500000]

fair_prices = []
autocall_ctn = []

for p in paths:
    # simulate p times
    price, ac_ctn = fair_price(p, S0, T, N, sigma_im, r, Ds, Pk, K, NOM, Dc, Pc, CP1, CP_DATE)
    fair_prices.append(price)
    autocall_ctn.append(ac_ctn)

fair_prices = np.array(fair_prices)
profit = NOM * P - fair_prices

print_ans(question, paths, fair_prices, profit, autocall_ctn, NOM, N)

(a) Monte Carlo Simulate   1000 paths: 	Product fair price: 98153.514 (98.154%); 	Investment bank profit: 1846.486 (1.846%); 	Num of Autocall:    858 (85.80%)
(b) Monte Carlo Simulate  10000 paths: 	Product fair price: 97987.239 (97.987%); 	Investment bank profit: 2012.761 (2.013%); 	Num of Autocall:   8560 (85.60%)
(c) Monte Carlo Simulate 100000 paths: 	Product fair price: 98161.469 (98.161%); 	Investment bank profit: 1838.531 (1.839%); 	Num of Autocall:  86285 (86.28%)
(d) Monte Carlo Simulate 500000 paths: 	Product fair price: 98133.458 (98.133%); 	Investment bank profit: 1866.542 (1.867%); 	Num of Autocall: 430722 (86.14%)


### 2. Repeat the above calculations with coupon per month CP set 2. What is the additional profit of the investment bank in each case?

In [58]:
np.random.seed(5718) # for reproduction

paths = [1000, 10000, 100000, 500000]

fair_prices = []
autocall_ctn = []

for p in paths:
    # simulate p times
    price, ac_ctn = fair_price(p, S0, T, N, sigma_im, r, Ds, Pk, K, NOM, Dc, Pc, CP2, CP_DATE)
    fair_prices.append(price)
    autocall_ctn.append(ac_ctn)

fair_prices = np.array(fair_prices)
profit = NOM * P - fair_prices

print_ans(question, paths, fair_prices, profit, autocall_ctn, NOM, N)

(a) Monte Carlo Simulate   1000 paths: 	Product fair price: 97581.474 (97.581%); 	Investment bank profit: 2418.526 (2.419%); 	Num of Autocall:    858 (85.80%)
(b) Monte Carlo Simulate  10000 paths: 	Product fair price: 97416.186 (97.416%); 	Investment bank profit: 2583.814 (2.584%); 	Num of Autocall:   8560 (85.60%)
(c) Monte Carlo Simulate 100000 paths: 	Product fair price: 97599.275 (97.599%); 	Investment bank profit: 2400.725 (2.401%); 	Num of Autocall:  86285 (86.28%)
(d) Monte Carlo Simulate 500000 paths: 	Product fair price: 97568.221 (97.568%); 	Investment bank profit: 2431.779 (2.432%); 	Num of Autocall: 430722 (86.14%)


### 3. Instead of using the given CP, find the level of CP such that the initial profit margin of the investment bank is 2.50%. You only need to generate one result based on 500000 Monte Carlo paths.

In [59]:
CPs = np.array([0.522,]) * 0.01

path = 500000
fair_prices = []
autocall_ctn = []
for cp in CPs:
    np.random.seed(5718) # for reproduction
    price, _ = fair_price(path, S0, T, N, sigma_im, r, Ds, Pk, K, NOM, Dc, Pc, cp, CP_DATE)
    fair_prices.append(price)
    autocall_ctn.append(ac_ctn)

fair_prices = np.array(fair_prices)
profits = NOM * P - fair_prices

for i in range(len(CPs)):
    p = profits[i]
    print(f'coupon rate: {CPs[i]*100:.8f}%, \tfair price {fair_prices[i]:.3f}, \tprofit: {p:.3f} ({p/NOM*100:.3f}%)')

coupon rate: 0.52200000%, 	fair price 97499.965, 	profit: 2500.035 (2.500%)


### 4. The above pricing procedure is an approximation (refer to the comments in the course notes). In real life, weekends and holidays need to be considered and these are non-business days. If the above procedure is modified by taking these into account, do you think that the coupon calculated to be higher or lower than CP calculated in question 3 above? Give a brief reason to explain your answer.

It depends on how does the ratio of num_days and total_days change. The profit of investment banks is equal to investors payment minus product fair price. If other parameters are unchanged and the proft keeps the same, the fair price must keep the same. The fair price of a product is the average payoff of the product. In the earily termination condition, the payoff includes accrude interest, i.e NOM * CP * num_days / total_days, it's clear that either higher ratio or coupoun rate will get higher payoff. Assume autocall is triggered at the same day in the two scenarios that non-business days are considered or not, to balance the change in those cases: (i) if the ratio of num_days and total_days decreases, the coupon rate should go higher; (ii) if the ratio of num_days and total_days increases, the coupon rate should go lower.

Assume there are 30 days in one month. There are at most 22 trading days in one month. Let X denote the number of days between the call date and the coupon date immediately preceding the call date, i.e. num_days; Let Y denote the number of days between the coupon dates immediately preceding and following the call date, i.e. total_days. The change of the ratio R = X / Y can discuss as follows:

Example Case 1: There is no non-business day before autocall and there are 22 trading days in this month, e.g. X = 1, Y = 30 -> Y' = 22. If an autocall is triggered, the payoff changes:

In [60]:
X = 1; Xp = 1; Y = 30; Yp = 22
print(f"The payoff of considering non-business days: X / Y = {X/Y:.3f}; \tThe payoff of considering business days only: Xp / Yp = {Xp/Yp:.3f} \tthe ratio increases.")

The payoff of considering non-business days: X / Y = 0.033; 	The payoff of considering business days only: Xp / Yp = 0.045 	the ratio increases.


Example Case 2: There are 22 trading days in this month, and today is the 21st trading day and 29th days, i.e., X = 29 -> Xp = 21, Y = 30 -> Y' = 22. If an autocall is triggered, the payoff changes: 

In [61]:
X = 29; Xp = 21; Y = 30; Yp = 22
print(f"The payoff of considering non-business days: X / Y = {X/Y:.3f}; \tThe payoff of considering business days only: Xp / Yp = {Xp/Yp:.3f} \tthe ratio decreases.")

The payoff of considering non-business days: X / Y = 0.967; 	The payoff of considering business days only: Xp / Yp = 0.955 	the ratio decreases.


In conclusion, we can't guarantee how will the ratio X / Y change. The actual change of CP depends on the ratio of num_days and total_days which is undetermined.